In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.compat.v1.losses import sparse_softmax_cross_entropy as sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [32]:
# Create a custom VGG-16 model with shorter depth, increased max pooling, and dropout
def modified_vgg16(input_shape=(224, 224, 3), num_classes=13):
    model = models.Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='valid', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten
    model.add(Flatten())

    #Dense layer
    model.add(Dense(256, activation='relu'))
    
    # Dropout for Regularization
    model.add(layers.Dropout(0.5))
    
    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Instantiate the modified model
modified_model = modified_vgg16(input_shape=(224, 224, 3), num_classes=13)

# Compile the model
modified_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Step 1: Store file paths and split into training and test
main_folder = "C:\\Dataset"

# List to store image file paths and corresponding labels
image_paths = []
labels = []

# Traverse through the directory structure
for brand_folder in os.listdir(main_folder):
    brand_path = os.path.join(main_folder, brand_folder)
    
    if os.path.isdir(brand_path):
        for image_file in os.listdir(brand_path):
            # Check if the file is an image (you might want to add more extensions)
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(brand_path, image_file)
                image_paths.append(image_path)
                labels.append(brand_folder)  # Assuming brand name is the label

# Convert labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Split into training and test sets
X_train_paths, X_test_paths, y_train, y_test = train_test_split(image_paths, y_encoded, test_size=0.2, random_state=42)

In [8]:
num_batches = 130

# Create a list to store the loaded batches
train = []

# Load all saved batches
for i in range(num_batches):
    # Define the file name or path for each batch
    load_path = f'C:\\Users\\arava\\OneDrive\\Desktop\\New folder\\X_train\\processed_batch_{i}.npy'
    
    # Load the batch and append it to the list
    loaded_batch1 = np.load(load_path)
    train.append(loaded_batch1)

In [9]:
num_batches = 33

# Create a list to store the loaded batches
test = []

# Load all saved batches
for i in range(num_batches):
    # Define the file name or path for each batch
    load_path = f'C:\\Users\\arava\\OneDrive\\Desktop\\New folder\\X_test\\processed_batch_{i}.npy'
    
    # Load the batch and append it to the list
    loaded_batch2 = np.load(load_path)
    test.append(loaded_batch2)

In [18]:
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)
loaded_y_train = np.load('y_train.npy')
loaded_y_test = np.load('y_test.npy')

In [25]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# File paths for processed image data batches
image_folder = 'C:\\Users\\arava\\OneDrive\\Desktop\\New folder\\X_train\\'
num_batches = 129

# Load processed image data batches
image_data_batches = [np.load(os.path.join(image_folder, f'processed_batch_{i}.npy')) for i in range(num_batches)]

# Load additional individual images batch
additional_images_batch = np.load(os.path.join(image_folder, 'processed_batch_129.npy'))

# Concatenate all image data
all_image_data = np.concatenate(image_data_batches + [additional_images_batch])

# Load labels from 'y_train.npy' without the image folder path
labels = np.load('y_train.npy')  # Adjust the path accordingly

# Convert labels to categorical format
labels_categorical = to_categorical(labels, num_classes=13) 


In [33]:
# Train the model
history = modified_model.fit(
    all_image_data, labels_categorical,
    epochs=5,  
    batch_size=32  
)

# Save the trained model weights and biases
modified_model.save_weights('model_weights.h5')

# Save the model architecture and configuration
with open('model_config.json', 'w') as json_file:
    json_file.write(modified_model.to_json())

# Save the training history for future analysis or plotting
np.save('training_history.npy', history.history)


Epoch 1/5
130/130 [==============================] - 700s 5s/step - loss: 6.6624 - accuracy: 0.1458
Epoch 2/5
130/130 [==============================] - 685s 5s/step - loss: 2.5262 - accuracy: 0.1649
Epoch 3/5
130/130 [==============================] - 687s 5s/step - loss: 2.5131 - accuracy: 0.1651
Epoch 4/5
130/130 [==============================] - 682s 5s/step - loss: 2.5101 - accuracy: 0.1651
Epoch 5/5
130/130 [==============================] - 681s 5s/step - loss: 2.5069 - accuracy: 0.1651
